# Train Diffusion Model
### Imports

In [1]:
try: 
    import librosa
except:
    !pip install librosa


#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
import torch.optim as optim

# Utils
import numpy as np
from numpy import ndarray
import logging

# Base Scripts
from Libraries.U_Net import *
from Libraries.Diffusion import *
from Libraries.Utils import *
from MainScripts.Conf import conf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 71.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 42.2 MB/s eta 0:00:00:00:0100:01


### Config
General

In [ ]:
remote_kernel: bool = True

logging_level: int = logging.INFO
model_name: str = "diffusion_v6"
full_model_path: str = path_to_remote_path("{}/{}".format(conf["paths"].model_path, model_name + ".pth"), remote_kernel)
checkpoint_freq: int = 10 #0 for no checkpoint saving
training_data_name: str = "training_full_low_res"

device = "cuda" if torch.cuda.is_available() else "cpu"
restart_training: bool = True #If True and model already exists optimizer and lr_scheduler are reset
learning_rate: float = 5e-4
epochs: int = 100
n_training_samples: int = 2000

logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

### Data Loading

In [3]:
file: ndarray = load_training_data(path_to_remote_path("{}/{}".format(conf["paths"].data_path, training_data_name + ".npy"), remote_kernel))[:n_training_samples, ...]
data_loader = create_dataloader(Audio_Data(file), conf["model"].batch_size)
logger.info(f"Data loaded with shape: {file.shape}")

2025-03-16 15:00:14,017 - INFO - Data loaded with shape: (2000, 224, 416)


### Model Creation
U-Net

In [4]:
u_net = Conv_U_NET(in_channels=1,
                    time_embed_dim=conf["model"].time_embed_dim, 
                    n_starting_filters=conf["model"].n_starting_filters, 
                    n_downsamples=conf["model"].n_downsamples, 
                    activation=nn.SiLU(), 
                    device=device
                ).to(device)

optimizer = optim.AdamW(u_net.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", factor=0.8, patience=5, threshold=0, threshold_mode="rel", cooldown=0,min_lr=1e-6)
start_epoch: int = 0

if os.path.exists(full_model_path):
    model = torch.load(full_model_path, map_location=device)
    u_net.load_state_dict(model["model"])
    if not restart_training:
        optimizer.load_state_dict(model["optim"])
        scheduler.load_state_dict(model["scheduler"])
        start_epoch = model.get("epoch", 0)
    logger.info(f"Model {model_name} loaded with {count_parameters(u_net)} Parameters")
else: 
    logger.info(f"Model {model_name} created with {count_parameters(u_net)} Parameters")

2025-03-16 15:00:16,052 - INFO - Model diffusion_v6 created with 69306945 Parameters


Diffusion

In [5]:
diffusion = Diffusion(model=u_net, 
                        noise_steps=conf["model"].diffusion_timesteps, 
                        noise_schedule="linear", 
                        input_dim=[conf["model"].batch_size, 1, file.shape[-2], file.shape[-1]],
                        device=device
                    )

#diffusion.visualize_diffusion_steps(x=torch.Tensor(file[:1]), noise_schedule=noise_schedule, device=device, n_spectograms=5)

### Train

In [ ]:
x = diffusion.train(epochs=epochs, 
                    data_loader=data_loader, 
                    loss_function=nn.MSELoss(),
                    optimizer=optimizer, 
                    lr_scheduler=scheduler, 
                    gradient_accum=conf["model"].gradient_accum,
                    checkpoint_freq=checkpoint_freq, 
                    model_path=full_model_path, 
                    start_epoch=start_epoch
                )
scatter_plot(x)

2025-03-16 15:00:16,345 - INFO - Training started on cuda
2025-03-16 15:03:22,151 - INFO - Epoch 001: Avg. Loss: 1.00999e+00 Remaining Time: 05h 06min 34s LR: 5.00000e-04
2025-03-16 15:06:29,032 - INFO - Epoch 002: Avg. Loss: 9.57198e-01 Remaining Time: 05h 04min 21s LR: 5.00000e-04
2025-03-16 15:09:36,556 - INFO - Epoch 003: Avg. Loss: 7.98481e-01 Remaining Time: 05h 01min 53s LR: 5.00000e-04
2025-03-16 15:12:44,214 - INFO - Epoch 004: Avg. Loss: 6.87070e-01 Remaining Time: 04h 59min 08s LR: 5.00000e-04
2025-03-16 15:15:54,119 - INFO - Epoch 005: Avg. Loss: 6.28756e-01 Remaining Time: 04h 56min 57s LR: 5.00000e-04
2025-03-16 15:19:02,340 - INFO - Epoch 006: Avg. Loss: 5.68245e-01 Remaining Time: 04h 54min 00s LR: 5.00000e-04
2025-03-16 15:22:12,331 - INFO - Epoch 007: Avg. Loss: 5.35603e-01 Remaining Time: 04h 51min 23s LR: 5.00000e-04
2025-03-16 15:25:20,357 - INFO - Epoch 008: Avg. Loss: 4.87659e-01 Remaining Time: 04h 48min 16s LR: 5.00000e-04
2025-03-16 15:28:28,649 - INFO - Epoch